In [ ]:
import numpy as np
import pandas as pd
import matplotlib

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Conhecendo o Dataset

A intenção era encontrar datas relacionados a movimentação de barcos obtidos pelo sistema AIS, disponiveis em todas as embarcações de médio e grande porte de carga e em todas as embarcações de passageiros.
  Os datas encontrados possuem dois comportamentos, o primeiro tipo parece uma foto, embarcações e suas posições no mar em um tempo especifico. Outro tipo é de trajetória clara, posições em função do tempo, esse segundo são bem mais escassos de encontrar e geralmente estão disponíveis em sites pagos. Como o VTExplotere e o MarineTraffic.com (de onde o data do trabalho “Open Data for Anomaly Detection in Maritime Surveillanc” foi retirado)
	Uma fonte confiável, do primeiro tipo, que retira seus dados a partir de dados da Guarda Costeira Americana foi encontrada no site MarineCadastre.gov, esse site contém registros em datas de 2009 a 2020.
  O data em questão, deste trabalho foi retirado desse site, ano de 2021, dia 1° de Janeiro, identificado por AIS_2020_01_01.



* MMSI -  Maritime Mobile Service Identity - Numero identificador exclusivo de estações relacionadas a embarcações.
* BaseDateTime - Data e hora da transmissão     
* LAT - Latitude             
* LON - Longitude             
* SOG - SpeedOver Ground (Velocidade sobre o solo)                
* COG -  course over ground (Direção sobre o solo)             
* Heading - Rumo/direção           
* VesselName -  Nome das embarcações       
* IMO -  número de identificação de navio              
* CallSign -          
* VesselType - Tipo das embarcações       
* Status -           
* Length - Comprimento           
* Width - Largura            
* Draft -             
* Cargo -             
* TranscieverClass - Classe do transmissor AIS (pode ser A ou B)

In [ ]:
# Carregando base de dados

url_database = '/content/drive/Shareddrives/CD - Embarcações/Amostra_2020_01_01.csv'
database = pd.read_csv(url_database, sep=',', header=None)
database.columns=['MMSI', 'BaseDateTime', 'LAT', 'LON', 'SOG', 'COG', 'Heading', 'VesselName', 'IMO', 'CallSign', 'VesselType', 'Status', 'Length', 'Width', 'Draft', 'Cargo', 'TranscieverClass']
data = database.copy(deep= True)
#df = data.drop(0)
data.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,4,5,6,10,11,12,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TranscieverClass
0,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TranscieverClass
1,367149340,2020-01-01T00:00:00,29.96476,-90.02724,1.3,10.0,16.0,SYDNEE TAYLOR,NaN,WDD4807,31,0,26,9,NaN,31,B
2,367687520,2020-01-01T00:00:00,30.20558,-91.03578,10.7,124.9,130.0,CHIPPEWA,NaN,WDI3361,31,0,22,NaN,NaN,NaN,B
3,367368170,2020-01-01T00:00:03,47.53785,-122.32833,0.6,46.5,141.0,SONJA H,NaN,WDE5536,31,0,18,6,NaN,32,B
4,367007980,2020-01-01T00:00:05,37.95154,-121.32682,0.0,-49.6,511.0,ANGIE M BRUSCO,IMO5111359,WDC3446,31,0,28,7,3.4,NaN,B


In [ ]:
database.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MMSI              45000 non-null  object 
 1   BaseDateTime      44977 non-null  object 
 2   LAT               44977 non-null  float64
 3   LON               44977 non-null  float64
 4   SOG               44977 non-null  float64
 5   COG               44977 non-null  float64
 6   Heading           44977 non-null  float64
 7   VesselName        41169 non-null  object 
 8   IMO               19094 non-null  object 
 9   CallSign          37472 non-null  object 
 10  VesselType        40479 non-null  float64
 11  Status            36163 non-null  float64
 12  Length            37926 non-null  float64
 13  Width             32274 non-null  float64
 14  Draft             11145 non-null  float64
 15  Cargo             13061 non-null  float64
 16  TranscieverClass  44955 non-null  object

In [ ]:
database

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TranscieverClass
0,367149340,2020-01-01T00:00:00,29.96476,-90.02724,1.3,10.0,16.0,SYDNEE TAYLOR,NaN,WDD4807,31.0,0.0,26.0,9.0,NaN,31.0,B
1,367687520,2020-01-01T00:00:00,30.20558,-91.03578,10.7,124.9,130.0,CHIPPEWA,NaN,WDI3361,31.0,0.0,22.0,NaN,NaN,NaN,B
2,367368170,2020-01-01T00:00:03,47.53785,-122.32833,0.6,46.5,141.0,SONJA H,NaN,WDE5536,31.0,0.0,18.0,6.0,NaN,32.0,B
3,367007980,2020-01-01T00:00:05,37.95154,-121.32682,0.0,-49.6,511.0,ANGIE M BRUSCO,IMO5111359,WDC3446,31.0,0.0,28.0,7.0,3.4,NaN,B
4,367538940,2020-01-01T00:00:05,30.00258,-93.22608,3.1,168.0,511.0,RITA ANN,NaN,WDG4670,31.0,0.0,21.0,NaN,NaN,NaN,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,376552000,2020-01-01T00:06:57,48.45736,-123.17011,-0.1,-49.6,356.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,B
44996,367387160,2020-01-01T00:07:00,38.17972,-85.90066,0.0,0.0,511.0,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,B
44997,368079520,2020-01-01T00:06:59,26.11726,-80.13867,0.0,186.0,216.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,A
44998,368014250,2020-01-01T00:07:00,30.28006,-91.15679,12.9,96.7,511.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,B


In [ ]:
#dados ausentes
print(database.VesselName.isna().sum())

SyntaxError: ignored

In [ ]:
database.groupby(["VesselName"]).agg(["count"])
teste = database.groupby(["VesselName"]).size().reset_index(name='counts')
teste[teste["counts"] >= 1]



,VesselName,counts
0,+BRAVA,2
1,0.C.D,2
2,1492,2
3,185 3,6
4,217,4
...,...,...
8839,ZUBE,3
8840,ZULLA C,2
8841,ZURI,3
8842,ZYANA K,4


In [ ]:
df_grouped =  database.groupby(['MMSI']).size().reset_index(name='count')
print (df_grouped)

             MMSI  count
0         1056261      7
1      1073709206      7
2         1193046      4
3       122292919      6
4        12345678      6
...           ...    ...
10411   941000944      3
10412   941202479      2
10413   941206243      1
10414   941206308      3
10415   941207025      1

[10416 rows x 2 columns]


In [ ]:
pip install folium

import folium
from folium import plugins

coordenadas=[]
for lat,lng in zip(data.LAT.values[:100],df.LON.values[:100]):
  coordenadas.append([lat,lng])

mapa = folium.Map(location=[30.0675,-89.9272], zoom_start= 4, min_lat=- 180, max_lat=0, min_lon= 80, max_lon=0, tiles='Stamen Toner')

mapa.add_child(plugins.HeatMap(coordenadas))
mapa

In [ ]:
import folium
from folium import plugins

coordenadas=[]
for lat,lng in zip(data.LAT.values[:100],df.LON.values[:100]):
  coordenadas.append([lat,lng])

In [ ]:
mapa = folium.Map(location=[30.0675,-89.9272], zoom_start= 4, min_lat=- 180, max_lat=0, min_lon= 80, max_lon=0, tiles='Stamen Toner')

In [ ]:
mapa.add_child(plugins.HeatMap(coordenadas))
mapa